In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from numpy.linalg import inv

import itertools
import utils.itertools_recipes

In [ ]:
size = 100
low = 1
hi = 5

In [ ]:
range_ends = np.r_[low, hi]

In [ ]:
θ_0 = np.random.uniform(0, 1)
θ_1 = np.random.uniform(0, 3)

solution = np.vectorize(lambda x: θ_0 + θ_1 * x)

In [ ]:
θ_0, θ_1

In [ ]:
xs = np.random.uniform(low, hi, size)
y_noise = np.random.randn(size)
ys = θ_0 + θ_1 * xs + y_noise 

In [ ]:
plt.plot(xs, ys, '.r')

In [ ]:
X = np.c_[np.ones(xs.size), xs]
Y = ys[:,None]

In [ ]:
# Commented in favor of more linear algebraic formula that is more compact and more general
# opt_θ_1 = (ys.mean() - (xs * ys).sum() / xs.sum()) / (xs.mean() - (xs ** 2).sum() / xs.sum())
# opt_θ_0 = ys.mean() - opt_θ_1 * xs.mean()

opt_θ_0, opt_θ_1 = (inv(X.T @ X) @ X.T @ Y)[:,0]

h = np.vectorize(lambda x: opt_θ_0 + opt_θ_1 * x)

In [ ]:
opt_θ_0, opt_θ_1

In [ ]:
print("squared error = ", (θ_0 - opt_θ_0) ** 2 + (θ_1 - opt_θ_1) ** 2)

In [ ]:
plt.plot(xs, ys, '.r')
plt.plot(range_ends, h(range_ends))
plt.plot(range_ends, solution(range_ends))

In [ ]:
vecJ = lambda θ: ((X @ θ.reshape(2, 1)  - Y) ** 2).sum() / xs.size
J = np.vectorize(lambda θ_0, θ_1: vecJ(np.array([θ_0, θ_1])))

In [ ]:
θ_0_range = np.r_[θ_0 - 1 : θ_0 + 1 : 100j]
θ_1_range = np.r_[θ_1 - 1 : θ_1 + 1 : 100j]
plt.contour(θ_1_range, θ_0_range, J(θ_0_range[:,None], θ_1_range[None,:]))

In [ ]:
dJ = lambda θ: 2 / xs.size * (X * (X @ θ.reshape(2, 1) - Y)).sum(axis=0)

In [ ]:
def batchGradDescent(start_θ, α = 0.01):
    current_θ = start_θ
    while True:
        yield current_θ
        current_θ = current_θ - α * dJ(current_θ)

In [ ]:
def optimize(α = 0.01):
    return batchGradDescent(start_θ=np.array([0, 0]), α=α)

In [ ]:
itertools_recipes.nth(optimize(), 20000)

In [ ]:
def progressJ(α, steps=40):
    return itertools_recipes.take(steps, map(vecJ, optimize(α=α)))

#plt.plot(progressJ(0.1), label='α = 0.1') # diverges
plt.plot(progressJ(0.07), label='α = 0.07')
plt.plot(progressJ(0.03), label='α = 0.03')
plt.plot(progressJ(0.01), label='α = 0.01')
plt.plot(progressJ(0.001), label='α = 0.001')

plt.legend()